In [6]:
#run pip install yfinance --user before running this file

import yfinance as yf
import pandas as pd

In [7]:
#ticker symbol
ticker = yf.Ticker(input('Enter ticker symbol: '))

#TODO: PRINT INFO FOR TICKER SYMBOL

In [8]:
#expiration dates
expDates = ticker.options

print("Here are the option chain dates")
for date in expDates:
  print(date)

Here are the option chain dates
2024-03-15
2024-03-22
2024-03-28
2024-04-05
2024-04-12
2024-04-19
2024-04-26
2024-05-17
2024-06-21
2024-07-19
2024-08-16
2024-09-20
2024-10-18
2024-11-15
2024-12-20
2025-01-17
2025-03-21
2025-06-20
2025-09-19
2025-12-19
2026-01-16
2026-06-18


In [9]:
#call and put option data for front week
opChain = ticker.option_chain(date = input('Enter desired expiration date: '))

callsOrPuts = False #True for calls, False for puts
df = ''
if input("Do you want calls or puts? Enter 'c' for calls, 'p' for puts: ") == 'c':
  df = pd.DataFrame(opChain.calls)
else:
  df = pd.DataFrame(opChain.puts)

In [26]:
strike = input("Enter the strike price: ")
mask = df['strike'] == float(strike)
print('This is the information for your contract: ')
print(df[mask].transpose())

Here are the available strike prices: 
This is the information for your contract: 
                                           1
contractSymbol           AAPL240517C00025000
lastTradeDate      2024-02-16 19:33:19+00:00
strike                                  25.0
lastPrice                             158.39
bid                                    145.4
ask                                   147.35
change                                   0.0
percentChange                            0.0
volume                                     2
openInterest                               1
impliedVolatility                   2.315434
inTheMoney                              True
contractSize                         REGULAR
currency                                 USD


In [36]:
def blackScholesDelta(option, cOrP):
  S = 180 #underlying price
  K = option[2] #strike
  r = 0.05 #risk free return 
  T = 10 #time to maturity
  vol = 0.5 #underlying vol
  
  delta = 0
  
  if not cOrP:
    return 1 - delta
  return delta

print(f"The delta for your option is: {blackScholesDelta(df[mask].head().to_numpy(), callsOrPuts)}")

[['AAPL240517C00025000' Timestamp('2024-02-16 19:33:19+0000', tz='UTC')
  25.0 158.39 145.4 147.35 0.0 0.0 2 1 2.3154338989257806 True 'REGULAR'
  'USD']]
The delta for your option is: 5
